In [1]:
# import necessary libraries
import util
import numpy as np
import sqlite3
import pandas as pd
import ast

from IPython.display import Image
from IPython.display import display

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

matplotlib.rc("figure", figsize=(8,6))
matplotlib.rc("axes", labelsize=16, titlesize=16)
matplotlib.rc("xtick", labelsize=14)
matplotlib.rc("ytick", labelsize=14)
matplotlib.rc("legend", fontsize=14)
matplotlib.rc("font", size=14)

In [2]:
# load the rating matrix
M = util.load_sparse_csr("imdb_data/rating.npz").toarray()
movie_names = np.load("imdb_data/movies.npy")
user_names = np.load("imdb_data/users.npy")

print M.shape, movie_names.shape, user_names.shape

(18060, 1603) (1603,) (18060,)


In [3]:
print user_names

[u' (borkoboardo)' u'surfs_up1976' u'HumanoidOfFlesh' ...,
 u'Kaya Ozkaracalar' u'LJ27' u'KHayes666']


In [4]:
mask = M > 0
print mask.shape, mask.sum(axis=0).shape, mask.sum(axis=1).shape
print np.sum(mask.sum(axis=0) > 5)
print np.sum(mask.sum(axis=1) > 5)

(18060, 1603) (1603,) (18060,)
504
358


In [5]:
new_rating_m, new_user_names, new_movie_names = util.rating_filter(M, user_names, movie_names, (5,5))

print new_rating_m.shape, new_user_names.shape, new_movie_names.shape

(18060,) (1603,)
Original #rating 25658
Remaining #user 358 #movie 504
(358, 504)
Remaining #rating 4629
Sparsity for the rating matrix : 2.57%
(358, 504) (358,) (504,)


In [7]:
train, valid, test, user_mask = util.split_dataset(new_rating_m)
print train.shape, np.sum(train>0)
print valid.shape, np.sum(valid>0)
print test.shape, np.sum(test>0)

Number of user delete 5
(353, 504) 2669
(353, 504) 1040
(353, 504) 911
